In [3]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [14]:
d =[
    {'fecha':'01/01/2019','distrito':'centro','id':'1','les':'HL'},
    {'fecha':'01/01/2019','distrito':'centro','id':'1','les':'HG'},
    {'fecha':'01/01/2019','distrito':'arganzuela','id':'1','les':'HL'},
    {'fecha':'02/01/2019','distrito':'centro','id':'1','les':'HL'},
    {'fecha':'02/01/2019','distrito':'centro','id':'1','les':'HL'},
    {'fecha':'02/01/2019','distrito':'moratalaz','id':'1','les':'HL'}
]
df = spark.createDataFrame(d,schema=None)


AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

In [2]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData.select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy').alias('fecha'), \
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [3]:
#Las columnas del Dataframe accRed quedan así
accRed.printSchema()

root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- lesividad: string (nullable = true)



In [4]:
# Eliminamos las filas de las personas ilesas
# Agrupamos por fecha, distrito, accidente y lesividad para contar 
# los registros y tener el total de afectados.
accGrouped=accRed.filter(accRed.lesividad != "IL")\
                .groupBy('fecha','distrito','idAccidente','lesividad')\
                .agg(func.count('lesividad').alias('victimas'))\
                .sort ('fecha','distrito','idAccidente','lesividad')

In [9]:
accRed.groupBy('fecha').agg( func.count('lesividad')).show()

+-----+----------------+
|fecha|count(lesividad)|
+-----+----------------+
| null|           30122|
+-----+----------------+



In [10]:
#Las columnas del Dataframe accRed quedan así
accGrouped.printSchema()

root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- lesividad: string (nullable = true)
 |-- victimas: long (nullable = false)



#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [12]:
#accGrouped.toPandas().to_csv('/home/ubuntu/Downloads/victimas_por_fecha_distrito_accidente_lesion.csv',sep=';')
accGrouped.toPandas().to_csv('/home/jose/Descargas/victimas_por_fecha_distrito_accidente_lesion.csv',sep=';')